# Multi-label classification -- p-classification loss

In [ ]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.optimize import check_grad
from scipy.special import expit as sigmoid

from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, make_scorer, label_ranking_loss

import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

import torch

In [ ]:
sys.path.append('src')
from evaluate import avgPrecisionK, evaluatePrecision, evaluateF1, evaluateRankingLoss, f1_score_nowarn, calcLoss
from datasets import create_dataset, dataset_names, nLabels_dict

In [ ]:
data_ix = 3

In [ ]:
dataset_name = dataset_names[data_ix]
nLabels = nLabels_dict[dataset_name]
print(dataset_name, nLabels)

In [ ]:
data_dir = 'data'
SEED = 918273645
fmodel_base = os.path.join(data_dir, 'pn-' + dataset_name + '-base.pkl')
fmodel_prec = os.path.join(data_dir, 'pn-' + dataset_name + '-prec.pkl')

Load data.

In [ ]:
X_train, Y_train = create_dataset(dataset_name, train_data=True, shuffle=True, random_state=SEED)
X_test,  Y_test  = create_dataset(dataset_name, train_data=False)

Feature normalisation.

In [ ]:
X_train_mean = np.mean(X_train, axis=0).reshape((1, -1))
X_train_std = np.std(X_train, axis=0).reshape((1, -1)) + 10 ** (-6)
X_train -= X_train_mean
X_train /= X_train_std
X_test  -= X_train_mean
X_test  /= X_train_std

In [ ]:
def print_dataset_info(X_train, Y_train, X_test, Y_test):
    N_train, D = X_train.shape
    K = Y_train.shape[1]
    N_test = X_test.shape[0]
    print('%-45s %s' % ('Number of training examples:', '{:,}'.format(N_train)))
    print('%-45s %s' % ('Number of test examples:', '{:,}'.format(N_test)))
    print('%-45s %s' % ('Number of features:', '{:,}'.format(D)))
    print('%-45s %s' % ('Number of labels:', '{:,}'.format(K)))
    avgK_train = np.mean(np.sum(Y_train, axis=1))
    avgK_test  = np.mean(np.sum(Y_test, axis=1))
    print('%-45s %.3f (%.2f%%)' % ('Average number of positive labels (train):', avgK_train, 100*avgK_train / K))
    print('%-45s %.3f (%.2f%%)' % ('Average number of positive labels (test):', avgK_test, 100*avgK_test / K))
    #print('%-45s %.4f%%' % ('Average label occurrence (train):', np.mean(np.sum(Y_train, axis=0)) / N_train))
    #print('%-45s %.4f%%' % ('Average label occurrence (test):', np.mean(np.sum(Y_test, axis=0)) / N_test))
    print('%-45s %.3f%%' % ('Sparsity (percent) (train):', 100 * np.sum(Y_train) / np.prod(Y_train.shape)))
    print('%-45s %.3f%%' % ('Sparsity (percent) (test):', 100 * np.sum(Y_test) / np.prod(Y_test.shape)))

In [ ]:
print('%-45s %s' % ('Dataset:', dataset_name))
print_dataset_info(X_train, Y_train, X_test, Y_test)

## p-classification loss

Multi-label learning with p-classification loss.

In [ ]:
def obj_pclassification_gpu(w, X, Y, C, p, weighting=True):
    """
        Objective with L2 regularisation and p-classification loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
            - p: constant for p-classification push loss
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(p >= 1)
    assert(C > 0)
    
    if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor
    
    w_ = torch.from_numpy(w).type(dtype)
    X_ = torch.from_numpy(X).type(dtype)
    Y_ = torch.from_numpy(Y).type(dtype)
    
    #W = w.reshape(K, D)  # reshape weight matrix
    W_ = w_.view(K, D)
    
    OneN = torch.ones(N).type(dtype)
    OneK = torch.ones(K).type(dtype)
    
    if weighting is True:
        #KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
        #KNegAll = K - KPosAll        # number of negative labels for each example, N by 1
        KPosAll = Y_.sum(dim=1)
        KNegAll = K - KPosAll
    else:
        #KPosAll = np.ones(N)
        #KNegAll = np.ones(N)
        KPosAll = OneN
        KNegAll = OneN
        
    #A_diag = np.divide(1, KPosAll)  # N by 1
    #P_diag = np.divide(1, KNegAll)  # N by 1
    #OneN = np.ones(N)  # N by 1
    #OneK = np.ones(K)  # K by 1
    A_diag = torch.div(OneN, KPosAll)
    P_diag = torch.div(OneN, KNegAll)
    
    #T1 = np.dot(X, W.T)  # N by K
    T1 = torch.matmul(X_, W_.t())
    
    #T1p = np.multiply(Y, T1)
    #T2 = np.multiply(Y, np.exp(-T1p))  # N by K
    #T3 = T2 * A_diag[:, None]  # N by K
    T1p = torch.mul(Y_, T1)
    T2 = torch.mul(Y_, torch.exp(-T1p))
    T3 = T2 * A_diag[:, None]
    
    #T1n = np.multiply(1-Y, T1)
    #T4 = np.multiply(1-Y, np.exp(p * T1n))  # N by K
    #T5 = T4 * P_diag[:, None]  # N by K
    T1n = torch.mul(1-Y_, T1)
    T4 = torch.mul(1-Y_, torch.exp(p * T1n))
    T5 = T2 * A_diag[:, None]
    
    #J = np.dot(w, w) * 0.5 / C + (np.dot(OneN, np.dot(T3, OneK)) + np.dot(OneN, np.dot(T5/p, OneK))) / N
    J = torch.matmul(w_, w_) * 0.5 / C + \
        torch.matmul(OneN, torch.matmul(T3, OneK)) / N + \
        torch.matmul(OneN, torch.matmul(T5/p, OneK)) / N
    
    #G = W / C + (np.dot(T3.T, -X) + np.dot(T5.T, X)) / N
    G = W_ / C + (torch.matmul(T3.t(), -X_) + torch.matmul(T5.t(), X_)) / N
    
    #return (J, G.ravel())
    return (J, G.view(-1).cpu().numpy())

In [ ]:
def obj_pclassification(w, X, Y, C, p, weighting=True):
    """
        Objective with L2 regularisation and p-classification loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
            - p: constant for p-classification push loss
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == K * D)
    assert(p >= 1)
    assert(C > 0)
    
    W = w.reshape(K, D)  # reshape weight matrix
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    if weighting is True:
        KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
        KNegAll = K - KPosAll        # number of negative labels for each example, N by 1
    else:
        KPosAll = np.ones(N)
        KNegAll = np.ones(N)
    A_diag = np.divide(1, KPosAll)  # N by 1
    P_diag = np.divide(1, KNegAll)  # N by 1
    
    OneN = np.ones(N)  # N by 1
    OneK = np.ones(K)  # K by 1
    T1 = np.dot(X, W.T)  # N by K
    
    T1p = np.multiply(Y, T1)
    T2 = np.multiply(Y, np.exp(-T1p))  # N by K
    T3 = T2 * A_diag[:, None]  # N by K
    
    T1n = np.multiply(1-Y, T1)
    T4 = np.multiply(1-Y, np.exp(p * T1n))  # N by K
    T5 = T4 * P_diag[:, None]  # N by K
    
    J = np.dot(w, w) * 0.5 / C + (np.dot(OneN, np.dot(T3, OneK)) + np.dot(OneN, np.dot(T5/p, OneK))) / N
    
    G = W / C + (np.dot(T3.T, -X) + np.dot(T5.T, X)) / N
    
    return (J, G.ravel())

In [ ]:
def obj_pclassification_loop(w, X, Y, C, p, weighting=True):
    """
        Objective with L2 regularisation and p-classification loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x L
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
            - p: constant for p-classification push loss
    """
    N, D = X.shape
    L = Y.shape[1]
    assert(w.shape[0] == L * D)
    assert(p >= 1)
    assert(C > 0)
    
    W = w.reshape(L, D)  # reshape weight matrix
    
    J = 0.0  # cost
    G = np.zeros_like(W)  # gradient matrix
    if weighting is True:
        nPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
        nNegAll = L - nPosAll        # number of negative labels for each example, N by 1
    else:
        nPosAll = np.ones(N)
        nNegAll = np.ones(N)
    
    for k in range(L):
        wk = W[k, :]
        Yk = Y[:, k]
        sPosVec = np.dot(X[Yk == 1, :], wk)      # Nk+ by 1
        sNegVec = np.dot(X[Yk == 0, :], wk)      # NK- by 1
        nPosVec = nPosAll[Yk == 1]               # Nk+ by 1
        nNegVec = nNegAll[Yk == 0]               # NK- by 1
        
        #nPosVec = np.sum(Y[Yk == 1, :], axis=1)  # Nk+ by 1
        #nNegVec = np.sum(Y[Yk == 0, :], axis=1)  # NK- by 1
        
        #nPosVec = np.sum(Y[Yk == 1, :], axis=1) + 0.01 # Nk+ by 1 with smoothing
        #nNegVec = np.sum(Y[Yk == 0, :], axis=1) + 0.01 # NK- by 1 with smoothing
        
        #nPosVec = np.ones_like(sPosVec) * N
        #nNegVec = np.ones_like(sNegVec) * N
        
        lossPos = np.divide(np.exp(-sPosVec), nPosVec)     # NK+ by 1
        lossNeg = np.divide(np.exp(p * sNegVec), nNegVec)  # NK- by 1
        
        J += np.sum(lossPos) + np.sum(lossNeg) / p
        #print(X[Yk == 0, :][0])
        #print(np.exp(np.dot(X[Yk == 0, :][0], wk)))
        
        GradPos = -X[Yk == 1, :] * lossPos[:, None]
        GradNeg =  X[Yk == 0, :] * lossNeg[:, None]
        
        G[k, :] = np.sum(GradPos, axis=0) + np.sum(GradNeg, axis=0)
                
    #J = 0.5 * C * np.dot(w, w) + J / N
    #G = C * W + G / N
    
    # be consistent with scikit-learn C = 1 / (N * \lambda)
    # normalise the objective J by dividing it C 
    J = np.dot(w, w) / (2.0 * C) + J / N
    G = W / C + G / N
    
    return (J, G.ravel())

Check gradient

In [ ]:
#w0 = 0.001 * np.random.randn(Y_train.shape[1] * X_train.shape[1])
#check_grad(lambda w: obj_pclassification(w, X_train, Y_train, C=1, p=8)[0], 
#           lambda w: obj_pclassification(w, X_train, Y_train, C=1, p=8)[1], w0)

In [ ]:
def cmp_loop_vec(func_loop, func_vec, X_train, Y_train, p=1):
    print('%15s %15s %15s %15s %15s' % ('C','J_Diff', 'J_loop', 'J_vec', 'G_Diff'))
    w0 = 0.001 * np.random.randn(Y_train.shape[1] * X_train.shape[1])
    for e in range(-6, 10):
        C = 10**(e)
        J,  G  = func_loop(w0, X_train, Y_train, C, p=p)
        J1, G1 = func_vec( w0, X_train, Y_train, C, p=p)
        Gdiff = G1 - G
        print('%15g %15g %15g %15g %15g' % (C, J1 - J, J, J1, np.dot(Gdiff, Gdiff)))

In [ ]:
#cmp_loop_vec(obj_pclassification_loop, obj_pclassification, X_train, Y_train, p=8)

In [ ]:
%%script false
C = 10
p = 1
w0 = np.random.rand(X_train.shape[1] * nLabels)
%lprun -f obj_pclassification check_grad(lambda w: obj_pclassification(w, X_train, Y_train, C, p)[0], \
                                         lambda w: obj_pclassification(w, X_train, Y_train, C, p)[1], w0)

In [ ]:
class MLC_pclassification(BaseEstimator):
    """All methods are necessary for a scikit-learn estimator"""
    
    def __init__(self, C=1, p=1, weighting=True):
        """Initialisation"""
        
        assert C >  0
        assert p >= 1
        self.C = C
        self.p = p
        self.weighting = weighting
        self.obj_func = obj_pclassification
        self.trained = False
        
    def fit(self, X_train, Y_train):
        """Model fitting by optimising the objective"""
        opt_method = 'L-BFGS-B' #'BFGS' #'Newton-CG'
        options = {'disp': 1, 'maxiter': 10**5, 'maxfun': 10**5} # , 'iprint': 99}
        sys.stdout.write('\nC: %g, p: %g, weighting: %s' % (self.C, self.p, self.weighting))
        sys.stdout.flush()
            
        N, D = X_train.shape
        K = Y_train.shape[1]
        #w0 = np.random.rand(K * D) - 0.5  # initial guess in range [-1, 1]
        w0 = 0.001 * np.random.randn(K * D)
        opt = minimize(self.obj_func, w0, args=(X_train, Y_train, self.C, self.p, self.weighting), \
                       method=opt_method, jac=True, options=options)
        if opt.success is True:
            self.W = np.reshape(opt.x, (K, D))
            self.trained = True
        else:
            sys.stderr.write('Optimisation failed')
            print(opt.items())
            self.trained = False
            
            
    def decision_function(self, X_test):
        """Make predictions (score is real number)"""
        
        assert self.trained is True, "Can't make prediction before training"
        D = X_test.shape[1]
        return np.dot(X_test, self.W.T)  # log of prediction score
        
    
    def predict(self, X_test):
        return self.decision_function(X_test)
    #    """Make predictions (score is boolean)"""   
    #    preds = sigmoid(self.decision_function(X_test))
    #    #return (preds >= 0)
    #    assert self.TH is not None
    #    return preds >= self.TH        
        
    # inherit from BaseEstimator instead of re-implement
    #
    #def get_params(self, deep = True):
    #def set_params(self, **params):

In [ ]:
def dump_results(predictor, X_train, Y_train, X_test, Y_test, rankingLoss=False):
    """
        Compute and save performance results
    """
    preds_train = predictor.decision_function(X_train)
    preds_test  = predictor.decision_function(X_test)
    
    print('Training set:')
    perf_dict_train = evaluatePrecision(Y_train, preds_train, verbose=1)
    print()
    print('Test set:')
    perf_dict_test = evaluatePrecision(Y_test, preds_test, verbose=1)
    
    if rankingLoss is True:
        print()
        print('Training set:')
        perf_dict_train.update(evaluateRankingLoss(Y_train, preds_train))
        print(label_ranking_loss(Y_train, preds_train))
        print()
        print('Test set:')
        perf_dict_test.update(evaluateRankingLoss(Y_test, preds_test))
        print(label_ranking_loss(Y_test, preds_test))

In [ ]:
def avgF1(Y_true, Y_pred):
    #THs = [0, 0.05, 0.10, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.45, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75]  # SPEN THs
    THs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    F1 = Parallel(n_jobs=-1)(delayed(f1_score_nowarn)(Y_true, Y_pred >= th, average='samples') for th in THs)
    bestix = np.argmax(F1)
    print('\nbest threshold: %g, best F1: %g, #examples: %g' % (THs[bestix], F1[bestix], Y_true.shape[0]))
    return F1[bestix]

In [ ]:
#C_set = [1e-3, 3e-3, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300]
C_set = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]
p_set = [1, 2, 4, 8]
parameters = [{'C': C_set, 'p': p_set, 'weighting': [True]}]
#scorer = {'Prec': make_scorer(avgPrecisionK)}
scorer = {'F1': make_scorer(avgF1)}

In [ ]:
if not os.path.exists(fmodel_prec):
    clf = GridSearchCV(MLC_pclassification(), parameters, scoring=scorer, cv=5, n_jobs=1, refit='F1')
    clf.fit(X_train, Y_train)
    #pkl.dump(clf, open(fmodel_prec, 'wb'))
else:
    clf = pkl.load(open(fmodel_prec, 'rb'))

In [ ]:
pkl.dump(clf, open(fmodel_prec, 'wb'))

In [ ]:
clf = MLC_pclassification(C=100, p=3)
clf.fit(X_train, Y_train)

In [ ]:
f1_score_nowarn(Y_test, clf.decision_function(X_test) >= 0.8, average='samples')

In [ ]:
f1_score_nowarn(Y_train, clf.decision_function(X_train) > 0.8, average='samples')

In [ ]:
dump_results(clf, X_train, Y_train, X_test, Y_test, rankingLoss=False)

In [ ]:
clf = pkl.load(open(fmodel_prec, 'rb'))

In [ ]:
clf.best_params_

In [ ]:
ysum = Y_train.sum(axis=1)

In [ ]:
ax = plt.subplot('111')
ax.hist(ysum, bins=20)
ax.set_yscale('log')

In [ ]:
f1_score_nowarn(Y_test, clf.decision_function(X_test) >= 0.8, average='samples')

In [ ]:
f1_score_nowarn(Y_train, clf.decision_function(X_train) > 0.9, average='samples')

In [ ]:
preds = clf.decision_function(X_test)

In [ ]:
np.max(preds)

In [ ]:
np.min(preds)

In [ ]:
#f1_score_nowarn(Y_test, sigmoid(preds) > 0.7, average='samples')

## Result analysis

In [ ]:
precisions_train = [avgPrecision(Y_train, preds_train, k) for k in range(1, nLabels+1)]
precisions_test  = [avgPrecision(Y_test,  preds_test,  k) for k in range(1, nLabels+1)]

In [ ]:
precisionK_train = avgPrecisionK(Y_train, preds_train)
precisionK_test  = avgPrecisionK(Y_test,  preds_test)

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(precisions_train, ls='--', c='r', label='Train')
plt.plot(precisions_test,  ls='-',  c='g', label='Test')
plt.plot([precisionK_train for k in range(nLabels)], ls='-', c='r', label='Train, Precision@K')
plt.plot([precisionK_test  for k in range(nLabels)], ls='-', c='g', label='Test, Precision@K')
plt.xticks(np.arange(nLabels), np.arange(1,nLabels+1))
plt.xlabel('k')
plt.ylabel('Precision@k')
plt.legend(loc='best')
plt.title('MLC w. p-classification Loss on ' + dataset_name + ' dataset')
plt.savefig(dataset_name + '_pc.svg')